In [6]:
import numpy as np
import pandas as pd

data = pd.read_csv(r'/Users/kittysu/Documents/Projects/PyFly/encoded_data.csv', index_col = None, header=0) 
data.drop('departure_date', axis=1, inplace=True)
data.drop_duplicates(inplace = True) 

data.tail()

,airfare,duration,num_stops,days_to_departure,day_of_week,departure_category,arrival_category,airline,day_of_year
31630,975,26.2,2,31,0,1,0,22,105
31631,975,26.8,2,31,0,1,0,22,105
31632,975,27.3,2,31,0,1,0,22,105
31633,975,28.3,2,31,0,1,0,22,105
31634,975,29.3,2,31,0,1,0,22,105


In [7]:
# BUY OR WAIT
data['buy'] = 0

# For each departure date
dates = data['day_of_year'].unique()
for d in dates:
    
    temp_df_1 = data[data['day_of_year'] == d]
    
    # For each purchase date
    days_to_depart = temp_df_1['days_to_departure'].unique()  
    
    for dd in days_to_depart:
        
        if (dd != 1):
            
            temp_df_2 = temp_df_1[temp_df_1['days_to_departure'] == dd]
            temp_min_2 = temp_df_2['airfare'].min()

            temp_df_3 = temp_df_1[temp_df_1['days_to_departure'] < dd]
            temp_min_3 = temp_df_3['airfare'].min()
        
            if (temp_min_2 <= temp_min_3):
                
                row = temp_df_2['airfare'].idxmin()
                data.at[row, 'buy'] = 1
                
        else:
            
            temp_df_2 = temp_df_1[temp_df_1['days_to_departure'] == dd]
            temp_min_2 = temp_df_2['airfare'].min()
            
            row = temp_df_2['airfare'].idxmin()
            data.at[row, 'buy'] = 1
            
# Calculate number of days to wait
data['days_to_wait'] = 0
data.reset_index(inplace=True, drop=True)

# For each row in data
rows = data.index
for r in rows:
    
    if (data.iloc[r]['buy'] == 0):
    
        d = data.iloc[r]['day_of_year']
        dd = data.iloc[r]['days_to_departure']
    
        temp_df_1 = data[data['day_of_year'] == d]

        temp_df_2 = temp_df_1[temp_df_1['days_to_departure'] < dd]
        temp_min_2 = temp_df_2['airfare'].min()
        
        if (temp_min_2 is not np.nan or temp_min_2 == temp_min_2):
            
            row = temp_df_2['airfare'].idxmin()
            ddd = data.iloc[row]['days_to_departure']
            data.at[r, 'days_to_wait'] = dd-ddd
            
data.tail()

,airfare,duration,num_stops,days_to_departure,day_of_week,departure_category,arrival_category,airline,day_of_year,buy,days_to_wait
24914,975,26.2,2,31,0,1,0,22,105,0,5
24915,975,26.8,2,31,0,1,0,22,105,0,5
24916,975,27.3,2,31,0,1,0,22,105,0,5
24917,975,28.3,2,31,0,1,0,22,105,0,5
24918,975,29.3,2,31,0,1,0,22,105,0,5


In [32]:
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor


# Split-out validation dataset
X = data[['airfare', 'duration', 'num_stops', 'days_to_departure', 'day_of_week', 'departure_category', 'arrival_category', 'airline', 'day_of_year']].values
y = data['days_to_wait'].values

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33, random_state=7)

# Linear regression
LR = LinearRegression(normalize=True)
LR.fit(X_train,y_train)
y_pred = LR.predict(X_test)

#LR.coef_

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


Mean Absolute Error: 3.513233497277137
Mean Squared Error: 19.5824618085096
Root Mean Squared Error: 4.425207544116954


In [33]:
# Logistic regression
LG = LogisticRegression(solver='liblinear', multi_class='ovr')
LG.fit(X_train,y_train)
y_pred = LG.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


Mean Absolute Error: 3.950510700389105
Mean Squared Error: 36.73650291828794
Root Mean Squared Error: 6.061064503722752


In [30]:
# Ridge regression
alphas = [1e-15, 1e-2, 1, 5]

for a in alphas:

    print('Alpha:',a)
    
    RR = Ridge(alpha=a,normalize=True)
    RR.fit(X_train,y_train)
    y_pred = RR.predict(X_test)

    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print(' ')
            

Alpha: 1e-15
Mean Absolute Error: 3.513233497277136
Mean Squared Error: 19.582461808509596
Root Mean Squared Error: 4.425207544116953
 
Alpha: 0.01
Mean Absolute Error: 3.498394887896388
Mean Squared Error: 19.561904528202707
Root Mean Squared Error: 4.422884186614285
 
Alpha: 1
Mean Absolute Error: 3.7499573595865447
Mean Squared Error: 23.484688971833442
Root Mean Squared Error: 4.84610038812997
 
Alpha: 5
Mean Absolute Error: 4.023664704558205
Mean Squared Error: 26.768008603417634
Root Mean Squared Error: 5.173780880885625
 


In [31]:
# Lasso regression
alphas = alphas = [1e-15, 1e-2, 1, 5]

for a in alphas:

    print('Alpha:',a)
    
    LS = Lasso(alpha=a,normalize=True, max_iter=1e5)
    LS.fit(X_train,y_train)
    y_pred = LS.predict(X_test)

    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print(' ')

Alpha: 1e-15
Mean Absolute Error: 3.5132334972770836
Mean Squared Error: 19.582461808509507
Root Mean Squared Error: 4.425207544116943
 
Alpha: 0.01
Mean Absolute Error: 3.842199609760853
Mean Squared Error: 24.599296703628788
Root Mean Squared Error: 4.959767807430988
 
Alpha: 1
Mean Absolute Error: 4.173707688175238
Mean Squared Error: 28.779447430353084
Root Mean Squared Error: 5.36464793163103
 
Alpha: 5
Mean Absolute Error: 4.173707688175238
Mean Squared Error: 28.779447430353084
Root Mean Squared Error: 5.36464793163103
 


In [34]:
# Elastic net
alphas = alphas = [1e-15, 1e-2, 1, 5]

for a in alphas:

    print('Alpha:',a)
    
    EN = ElasticNet(fit_intercept=True, alpha=a)
    EN.fit(X_train,y_train)
    y_pred = EN.predict(X_test)

    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print(' ')
    

Alpha: 1e-15
Mean Absolute Error: 3.5132334972771355
Mean Squared Error: 19.582461808509596
Root Mean Squared Error: 4.425207544116953
 
Alpha: 0.01
Mean Absolute Error: 3.5134229218809865
Mean Squared Error: 19.585356513289106
Root Mean Squared Error: 4.425534601976253
 
Alpha: 1
Mean Absolute Error: 3.4926535949957596
Mean Squared Error: 19.59478028735802
Root Mean Squared Error: 4.426599178529497
 
Alpha: 5
Mean Absolute Error: 3.4577538467009385
Mean Squared Error: 19.95818646226681
Root Mean Squared Error: 4.467458613380409
 


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [45]:
# CALCULATE TOTAL SAVINGS AND LOSSES
predictions = LR.predict(X)

data['predicted_wait'] = (np.floor(predictions)).tolist()
data.loc[data['predicted_wait'] < 0, 'predicted_wait'] = 0

total_savings = 0
total_losses = 0

# For each row in data
rows = data.index
for r in rows:
    
    d = data.iloc[r]['day_of_year']
    dd = data.iloc[r]['predicted_wait']
    
    temp_df_1 = data[data['day_of_year'] == d]
    
    temp_df_2 = temp_df_1[temp_df_1['days_to_departure'] < dd]
    temp_min_2 = temp_df_2['airfare'].min()
    
    temp_min_3 = data.iloc[r]['airfare']
    
    if (temp_min_2 is not np.nan or temp_min_2 == temp_min_2):
        
        if (temp_min_2 < temp_min_3):
        
            saving = temp_min_3 - temp_min_2
            total_savings += saving
        
        else:
            
            loss = temp_min_2 - temp_min_3
            total_losses += loss
            
data.tail()

,airfare,duration,num_stops,days_to_departure,day_of_week,departure_category,arrival_category,airline,day_of_year,buy,days_to_wait,predicted_wait
24914,975,26.2,2,31,0,1,0,22,105,0,5,10.0
24915,975,26.8,2,31,0,1,0,22,105,0,5,10.0
24916,975,27.3,2,31,0,1,0,22,105,0,5,10.0
24917,975,28.3,2,31,0,1,0,22,105,0,5,10.0
24918,975,29.3,2,31,0,1,0,22,105,0,5,10.0


In [50]:
print('Savings per flight', total_savings/24919)
print('Losses per flight', total_losses/24919)

Savings per flight 54.555720534531886
Losses per flight 11.894297523977688
